In [16]:
import cv2 # OpenCV for image processing
import numpy as np # Numpy for matrix operations
import os # OS for file operations
import glob # Glob for file operations

Preprocessing

In [17]:
def preprocessing(image_path: str):
    # Load the image and check if it was successfully loaded
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Cannot open image: {image_path}")

    # Calculate the scaling factor based on a maximum dimension of 800 pixels
    h, w = image.shape[:2]
    max_dim = max(h, w)
    scale_factor = 800 / max_dim
    resized = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor)

    # Convert the resized image to grayscale
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur with a 7x7 kernel to smooth out the image
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)

    # Apply adaptive thresholding using the MEAN method with a block size of 15 and constant 3
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY_INV, 15, 3)

    # Construct an output directory using the base name of the image file
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_dir = os.path.join("output", base_name)
    os.makedirs(output_dir, exist_ok=True)

    # Save the thresholded image into the output directory
    output_path = os.path.join(output_dir, "thresh_img.jpg")
    cv2.imwrite(output_path, thresh)

    return resized, thresh, scale_factor, base_name

Detecting Edges

In [18]:

def edge_detection(image, binary_img, scale_factor, filename):
    # Locate contours in the binary (thresholded) image
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    valid_contours = []

    # Evaluate each contour based on circularity and area criteria
    for cnt in contours:
        perimeter = cv2.arcLength(cnt, True)
        if perimeter > 0:
            # Compute circularity: 4*pi*Area/(Perimeter^2)
            circ_value = 4 * np.pi * (cv2.contourArea(cnt) / (perimeter ** 2))
            # Adjusted thresholds: circularity between 0.75 and 1.3 and area larger than 600*(scale_factor^2)
            if 0.75 < circ_value < 1.3 and cv2.contourArea(cnt) > 600 * (scale_factor ** 2):
                valid_contours.append(cnt)

    # Outline valid contours on the original image in green with a thickness of 2 pixels
    cv2.drawContours(image, valid_contours, -1, (0, 255, 0), 2)

    # Define output directory using the filename without its extension
    base_name = os.path.splitext(filename)[0]
    output_dir = os.path.join("output", base_name)
    os.makedirs(output_dir, exist_ok=True)
    cv2.imwrite(os.path.join(output_dir, "DetectedEdges.jpg"), image)

    return valid_contours

Segmentation

In [19]:
def segmentation(image, detected_contours, image_name):
    segmented_coins = []
    # Create output directory for segmented coins
    output_folder = "output"
    image_output_folder = os.path.join(output_folder, image_name.split('.')[0])  # Extract image name without extension
    os.makedirs(image_output_folder, exist_ok=True)

    for i, contour in enumerate(detected_contours):
        # Create a circular mask for each coin
        (x, y), radius = cv2.minEnclosingCircle(contour)
        center, radius = (int(x), int(y)), int(radius)
        mask = np.zeros_like(image, dtype=np.uint8)
        cv2.circle(mask, center, radius, (255, 255, 255), -1)
        
        # Extract the coin segment using bitwise AND
        coin_segment = cv2.bitwise_and(image, mask)[center[1] - radius:center[1] + radius, center[0] - radius:center[0] + radius]
        segmented_coins.append(coin_segment)
        
        # Save segmented coin directly
        cv2.imwrite(os.path.join(image_output_folder, f"Segmented_Coin_{i + 1}.jpg"), coin_segment)
    
    return segmented_coins


In [20]:
input_folder = "input"
image_paths = glob.glob(os.path.join(input_folder, "*.jpg"))
print("Found images:", image_paths)

for path in image_paths:
    image, threshold, scale_factor, image_name = preprocessing(path)
    detected_contours = edge_detection(image, threshold, scale_factor, image_name)
    segmented_coins = segmentation(image, detected_contours, image_name)
    
    # Moved count_coin function here
    def count_coin(detected_contours, segmented_coins):
        return (len(detected_contours), len(segmented_coins))

    detected, segmented = count_coin(detected_contours, segmented_coins)
    print(f"Image: {image_name} - Detected Coins: {detected}, Segmented Coins: {segmented}")


Found images: ['input/coin2.jpg', 'input/coin9.jpg', 'input/coin4.jpg']
Image: coin2 - Detected Coins: 2, Segmented Coins: 2
Image: coin9 - Detected Coins: 6, Segmented Coins: 6
Image: coin4 - Detected Coins: 4, Segmented Coins: 4
